In [2]:
import pandas as pd
import sqlite3

In [3]:
# Load the CSV file into a Pandas DataFrame
file_name = 'store_product_dataset.csv'  # Replace with your CSV file name
df = pd.read_csv(file_name)

# Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Load the DataFrame into the SQLite database
df.to_sql('store_product_dataset', conn, index=False, if_exists='replace')


1700

In [ ]:
#Data cleaning requirements:

Column Name	Criteria

product_id	Nominal. The unique identifier of the product.Missing values are not possible due to the database structure.

product_type	Nominal. The product category type of the product, one of 5 values (Produce, Meat, Dairy, Bakery, Snacks). Missing values should be replaced with “Unknown”.

brand	Nominal. The brand of the product. One of 7 possible values. Missing values should be replaced with “Unknown”.

weight	Continuous. The weight of the product in grams. This can be any positive value, rounded to 2 decimal places.Missing values should be replaced with the overall median weight.

price	Continuous. The price the product is sold at, in US dollars. This can be any positive value, rounded to 2 decimal places. Missing values should be replaced with the overall median price.

average_units_sold	Discrete. The average number of units sold each month. This can be any positive integer value. Missing values should be replaced with 0.

year_added	Nominal. The year the product was first added to FoodYum stock. Missing values should be replaced with last year (2022).

stock_location	Nominal. The location that stock originates. This can be one of four warehouse locations, A, B, C or D. Missing values should be replaced with “Unknown”.

In [4]:
# Query for data cleaning of columns concerned
query = """

SELECT 
    product_id, 
    product_type, 
    REPLACE(brand, '-', 'Unknown') AS brand, 
    ROUND(CAST(REPLACE(weight, ' grams', '') AS decimal), 2) AS weight, 
    ROUND(CAST(price AS decimal), 2) AS price, 
    average_units_sold, 
    COALESCE(year_added, '2022') AS year_added, 
    UPPER(stock_location) AS stock_location
FROM store_product_dataset;

"""

result = pd.read_sql(query, conn)
result

,product_id,product_type,brand,weight,price,average_units_sold,year_added,stock_location
0,1,Bakery,TopBrand,602.61,11.00,15,2022,C
1,2,Produce,SilverLake,478.26,8.08,22,2022.0,C
2,3,Produce,TastyTreat,532.38,6.16,21,2018.0,B
3,4,Bakery,StandardYums,453.43,7.26,21,2021.0,D
4,5,Produce,GoldTree,588.63,7.88,21,2020.0,A
...,...,...,...,...,...,...,...,...
1695,1696,Meat,TastyTreat,503.99,14.08,25,2017.0,A
1696,1697,Meat,GoldTree,526.89,16.13,25,2016.0,D
1697,1698,Bakery,YumMie,583.85,7.05,16,2021.0,A
1698,1699,Produce,TopBrand,441.64,8.10,19,2019.0,A


In [ ]:
#Requested data
Meat and dairy products where the average units sold was greater than 10.

In [9]:
#Query for requested data

query = """

SELECT product_id, price, average_units_sold
FROM store_product_dataset
WHERE product_type IN ('Meat','Dairy') AND average_units_sold > 10
order by average_units_sold DESC

"""

result = pd.read_sql(query, conn)
result

,product_id,price,average_units_sold
0,138,11.79,31
1,189,12.92,31
2,307,12.52,31
3,480,12.10,31
4,510,12.38,31
...,...,...,...
693,1547,12.96,20
694,1555,13.38,20
695,1561,13.05,20
696,1604,12.81,20
